## Demo of LLM function calling.

```bash
uv pip install numpydoc
```

- https://docs.litellm.ai/docs/completion/function_call

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
import json
import os
import random
import re
import warnings
from textwrap import dedent
from typing import Dict

import litellm
import openai

# from docstring_parser import parse
from faker import Faker
from gait import (
    a_message,
    function_to_tool,
    pydantic_to_tool,
    s_message,
    t_message,
    u_message,
)
from jupyprint import jupyprint
from litellm.utils import function_to_dict
from pydantic import BaseModel, Field
from rich.pretty import pprint

In [ ]:
litellm.drop_params = True
# litellm._turn_on_debug()  # 👈 this is the 1-line change you need to make

In [ ]:
# print(litellm.supports_function_calling(model="ollama_chat/llama3.2:latest"))
# print(litellm.supports_function_calling(model="azure/gpt-4o-mini"))
# print(litellm.supports_function_calling(model="ollama_chat/qwen2:7b-instruct-q8_0"))
# print(litellm.supports_function_calling(model="ollama_chat/phi4:14b-q8_0"))

In [ ]:
fake = Faker()

In [ ]:
def get_lat_lon(location: str) -> Dict:
    """Get the longitude and latitude of a given location.

    :param location: Can be a place, city, state, zipcode, state or country.
    :return: dict with location information.
    """
    return {
        "location": self.location,
        "longitude": float(lon),
        "latitude": float(lat),
    }

In [ ]:
pprint(function_to_tool(get_lat_lon), expand_all=True)

In [ ]:
class GetLatLon(BaseModel):
    """Get the latitude and longitude of a given location."""

    location: str = Field(
        ...,
        description="A location, can be a place, city, state, zipcode, state or country.",
    )

    def __call__(self, *args, **kwargs):
        lon = fake.longitude()
        lat = fake.latitude()
        return {
            "location": self.location,
            "longitude": float(lon),
            "latitude": float(lat),
        }

In [ ]:
# pprint(GetLatLon(location=fake.city())(), expand_all=True)

In [ ]:
# pprint(openai.pydantic_function_tool(GetLatLon), expand_all=True)

In [ ]:
# pprint(pydantic_to_tool(GetLatLon), expand_all=True)

In [ ]:
class GetRoute(BaseModel):
    """Get the route between a starting latitude/longitude location and an ending latitude/longitude location."""

    lon1: float = Field(
        ...,
        description="The route starting longitude.",
    )
    lat1: float = Field(
        ...,
        description="The route starting latitude.",
    )
    lon2: float = Field(
        ...,
        description="The route ending longitude.",
    )
    lat2: float = Field(
        ...,
        description="The route ending latitude.",
    )

    def __call__(self, *args, **kwargs):
        lon = fake.longitude()
        lat = fake.latitude()

        if "scratchpad" in kwargs:
            kwargs["scratchpad"]["GetRoute"] = {"lon": lon, "lat": lat}

        return {
            "route": f"{self.lat1},{self.lon1} ---> {self.lat2},{self.lon2}",
        }

In [ ]:
class GetCurrentTemperature(BaseModel):
    """Get the current temperature at a given location."""

    location: str = Field(
        ...,
        description="A location, can be a place, city, state, zipcode, state or country.",
    )
    celsius_or_fahrenheit: str = Field(
        ...,
        description="The temperature in either 'C' for Celsius, or 'F' for Fahrenheit.",
    )

    def __call__(self, *args, **kwargs):
        temp = random.uniform(-5, 40)
        return {
            self.location: f"{temp:.1f}{self.celsius_or_fahrenheit}",
        }

In [ ]:
base_models = [
    GetCurrentTemperature,
    GetLatLon,
    GetRoute,
]

## Convert BaseModels to tools and create LUT of name to BaseModel.

In [ ]:
tools = [pydantic_to_tool(_) for _ in base_models]

tool_dict = {_.__name__: _ for _ in base_models}

In [ ]:
fake = Faker()

In [ ]:
system = dedent(
    """
You are an AI expert in geo-spatial data analysis with access to specialized geo-spatial tools.
Your task is to answer a user’s question, denoted as >>>question<<<, related to geo-spatial data.
You will operate in a loop, alternating between reasoning about the problem and acting with tools as needed.
At the end of the loop, you must output a clear, accurate, and well-supported answer.

Follow these guidelines to complete your task using the ReAct (Reasoning + Acting) pattern:
- **Reason**: Break down the >>>question<<< into logical steps. Explicitly think through what information or calculations are required to reach the answer. Document your reasoning before taking any action.
- **Act**: Use the appropriate geo-spatial tools to gather data, perform analysis, or compute results based on your reasoning. When calling tools:
    - ALWAYS provide the correct, specific arguments required by the tool (e.g., "40.7128, -74.0060" for coordinates, not "lat, lon").
    - Use explicit values rather than placeholders or variable names.
    - NEVER repeat a tool call with identical arguments if it was already executed; reuse the prior result instead.
- Alternate between reasoning and acting as needed to refine your approach and solve the problem systematically.
- If the >>>question<<< is unclear, reason through possible interpretations, make reasonable assumptions based on geo-spatial context, and state them in your response.
- Before finalizing your answer, review your reasoning and tool outputs to ensure accuracy and relevance to the >>>question<<<.

Begin now! For each iteration:
1. **Reason**: Explain your next step or hypothesis.
2. **Act**: Call the necessary tool(s) or process the data.
3. Repeat until the task is solved.

If you solve the task correctly, you will receive a virtual reward of $1,000,000.
"""
).strip()

In [ ]:
prompt = dedent(
    f"""
>>>What's the route between {fake.city()} and {fake.city()}? And what is the temperature at each location?<<<
"""
).strip()


messages = [
    s_message(system),
    u_message(prompt),
]

In [ ]:
# for _ in messages:
#     pprint(_, expand_all=True)

## Start the loop.

In [ ]:
response = litellm.completion(
    # model = "huggingface/MadeAgents/Hammer2.1-7b",
    # model="ollama_chat/qwen2:7b-instruct-q8_0",
    # model="ollama_chat/qwen3:8b",
    # model="ollama_chat/llama3.2",
    model="azure/gpt-4.1-mini",
    api_base=os.environ["AZURE_API_URL"] + "/gpt-4.1-mini",
    messages=messages,
    tools=tools,
    # tool_choice="auto",  # auto is default, but we'll be explicit
    # parallel_tool_calls=False,
    temperature=0.0,
    top_p=1.0,
    n=1,
)

In [ ]:
pprint(
    response.choices[0],
    expand_all=True,
)

print(response.choices[0].finish_reason)

## Append response message to message history.

In [ ]:
messages.append(response.choices[0].message)

## See if there is a message content.

In [ ]:
jupyprint(response.choices[0].message.content)

## Get tool calls and call the referenced function.

In [ ]:
for tool_call in response.choices[0].message.tool_calls or []:
    pprint(tool_call, expand_all=True)
    func_name = tool_call.function.name
    func_args = json.loads(tool_call.function.arguments)
    func = tool_dict[func_name]
    # print(func_name)
    # pprint(func_args, expand_all=True)
    messages.append(
        t_message(
            json.dumps(func(**func_args)()),
            name=func_name,
            tool_call_id=tool_call.id,
        )
    )